In [62]:
import numpy as np
from transformers import AutoTokenizer
import transformers
from datasets import Dataset
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForMultipleChoice, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [52]:
raw = np.load('SP-train.npy', allow_pickle=True)
train, dev = train_test_split(raw, test_size = 0.1, random_state=42)

In [41]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = BertForMultipleChoice.from_pretrained("google-bert/bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return(len(self.data))
    def __getitem__(self, ndx):
        ex = self.data[ndx]
        id = ex['id']
        question = ex['question']
        answer = ex['answer']
        distractor1 = ex['distractor1']
        distractor2 = ex['distractor2']
        distractor_unsure = ex['distractor(unsure)']
        label = ex['label']
        choice_list = ex['choice_list']
        choice_order = ex['choice_order']

        qs = [question] * 4
        inputs = self.tokenizer(qs, choice_list, truncation=True, padding='max_length', max_length = self.max_len)
        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)  # Use label as the index of the correct choice
        }


In [59]:
max_len = 512
batch_size = 16
epochs = 1
lr = 2e-4
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)

In [60]:
trainCustomData = CustomDataset(train, tokenizer, 512)
trainBatchedData = DataLoader(trainCustomData, batch_size=batch_size, shuffle=True)
devCustomData = CustomDataset(dev, tokenizer, 512)
devBatchedData = DataLoader(devCustomData, batch_size=batch_size, shuffle=True)



In [61]:
for e in tqdm(range(epochs)):
    model.train()
    for batch in trainBatchedData:
        inputIds = batch['input_ids']
        attentionMask = batch['attention_mask']
        labels = batch['labels']
        optimizer.zero_grad()
        outputs = model(inputIds, attention_mask=attentionMask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        

KeyboardInterrupt: 

In [37]:
# customData = CustomDataset(raw, tokenizer, 512)
# print(len(customData[0]['input_ids']))
# print(type(customData[0]['input_ids'][0]))
# print(tokenizer.decode(torch.tensor(customData[56]['input_ids'][0]), skip_special_tokens = True))

4
<class 'list'>
each of the 30 participants in the masquerade had to wear a unique hat to distinguish themselves from one another. the host, however, only tallied 29 when he counted the number of hats to determine attendance. all 30 persons had signed their names on the spreadsheet, which confused him. he repeated the count. there are still 29. how is that even doable? one person had a pretty beautiful hat.
